In [6]:
# Author: Soroush Arabi
# All rights reserved
# ============================
# Libraries
# ============================
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import logging
import importlib

# Suppress logging outputs in the notebook
logging.getLogger().setLevel(logging.WARNING)

%matplotlib inline
# ============================
# Import and Reload SpecgridAnalyser Class
# ============================
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)

try:
    import specgrid_processor_V4
    importlib.reload(specgrid_processor_V4)  # Reload to ensure the latest version is loaded
    from specgrid_processor_V4 import SpecgridAnalyser
except ImportError as e:
    print(f"Failed to import SpecgridAnalyser: {e}")
    raise

# ============================
# Set the File Path
# ============================
 
specgrid_path = r"grid_data\Grid_Spectroscopy001.3ds"   # Replace the file path

# ============================
# Verify the file exists
if not os.path.isfile(specgrid_path):
    print(f"File not found: {specgrid_path}")
else:
    print(f"File found: {specgrid_path}")

    # Initialize the SpecgridAnalyser
    analyser = SpecgridAnalyser(specgrid_path)

    # ============================
    # Initialize Widgets
    # ============================
    # Define bias window range text inputs based on bias range
    bias_min_value = analyser.bias.min() * 1e3  # Convert to mV
    bias_max_value = analyser.bias.max() * 1e3  # Convert to mV

    bias_min_input = widgets.FloatText(
        value=bias_min_value,
        description="Bias Min (mV):",
        layout=widgets.Layout(width="200px"),
        style={"description_width": "initial"},
    )

    bias_max_input = widgets.FloatText(
        value=bias_max_value,
        description="Bias Max (mV):",
        layout=widgets.Layout(width="200px"),
        style={"description_width": "initial"},
    )

    # Savitzky-Golay Filter Parameters
    window_length_slider = widgets.IntSlider(
        min=3,
        max=21,
        step=1,  # Ensures window_length is odd
        value=5,
        description="Window Length (mV):",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="400px"),
        continuous_update=False,
    )

    polyorder_slider = widgets.IntSlider(
        min=1,
        max=5,
        step=1,
        value=2,
        description="Polyorder:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="400px"),
        continuous_update=False,
    )

    # Peak Detection Parameters
    threshold_slider = widgets.FloatSlider(
        min=0.0,
        max=1.0,
        step=0.01,
        value=0.05,
        description="Peak Threshold (a. u.):",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="400px"),
        continuous_update=False,
    )

    min_distance_slider = widgets.IntSlider(
        min=1,
        max=30,
        step=1,
        value=2,
        description="Min Distance Between Peaks (mV):",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="400px"),
        continuous_update=False,
    )

    # Apply Gaussian Filter Checkbox
    apply_gaussian_filter_checkbox = widgets.Checkbox(
        value=False,
        description='Apply Gaussian Filter',
        style={"description_width": "initial"},
        layout=widgets.Layout(width="400px"),
    )

    # Subtract Linear Fit Checkbox
    subtract_linear_fit_checkbox = widgets.Checkbox(
        value=False,
        description='Subtract Linear Fit',
        style={"description_width": "initial"},
        layout=widgets.Layout(width="400px"),
    )

    # Figure Size Sliders
    fig_width_slider = widgets.FloatSlider(
        min=4,
        max=10,
        step=0.5,
        value=5,
        description="Figure Width:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="400px"),
        continuous_update=False,
    )

    fig_height_slider = widgets.FloatSlider(
        min=1,
        max=8,
        step=0.5,
        value=4,
        description="Figure Height:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="400px"),
        continuous_update=False,
    )

    # Saving Options
    save_grid_csv_checkbox = widgets.Checkbox(
        value=False,
        description='Save Grid Data as CSV',
        style={"description_width": "initial"},
        layout=widgets.Layout(width="400px"),
    )

    save_average_dIdV_csv_checkbox = widgets.Checkbox(
        value=False,
        description='Save Average dI/dV as CSV',
        style={"description_width": "initial"},
        layout=widgets.Layout(width="400px"),
    )

    save_histogram_csv_checkbox = widgets.Checkbox(
        value=False,
        description='Save Histogram as CSV',
        style={"description_width": "initial"},
        layout=widgets.Layout(width="400px"),
    )

    save_grid_pdf_checkbox = widgets.Checkbox(
        value=False,
        description='Save Grid Plot as PDF',
        style={"description_width": "initial"},
        layout=widgets.Layout(width="400px"),
    )

    save_average_dIdV_pdf_checkbox = widgets.Checkbox(
        value=False,
        description='Save Average dI/dV Plot as PDF',
        style={"description_width": "initial"},
        layout=widgets.Layout(width="400px"),
    )

    save_histogram_pdf_checkbox = widgets.Checkbox(
        value=False,
        description='Save Histogram Plot as PDF',
        style={"description_width": "initial"},
        layout=widgets.Layout(width="400px"),
    )

    # Output widgets to display plots
    output_specgrid = widgets.Output()
    output_average_spectrum = widgets.Output()
    output_histogram = widgets.Output()

    # ============================
    # Define the Interactive Plotting Function
    # ============================
    def update_plots(change=None):
        bias_min_mV = bias_min_input.value
        bias_max_mV = bias_max_input.value
        subtract_linear_fit = subtract_linear_fit_checkbox.value
        apply_gaussian_filter = apply_gaussian_filter_checkbox.value
        window_length = window_length_slider.value
        polyorder = polyorder_slider.value
        threshold = threshold_slider.value
        min_distance = min_distance_slider.value
        fig_width = fig_width_slider.value
        fig_height = fig_height_slider.value
        save_grid_csv = save_grid_csv_checkbox.value
        save_average_dIdV_csv = save_average_dIdV_csv_checkbox.value
        save_histogram_csv = save_histogram_csv_checkbox.value
        save_grid_pdf = save_grid_pdf_checkbox.value
        save_average_dIdV_pdf = save_average_dIdV_pdf_checkbox.value
        save_histogram_pdf = save_histogram_pdf_checkbox.value

        bias_min = bias_min_mV * 1e-3  # Convert to V
        bias_max = bias_max_mV * 1e-3  # Convert to V

        # Apply or Reset Linear Fit Subtraction
        if subtract_linear_fit:
            analyser.subtract_linear_fit()
        else:
            analyser.reset_linear_fit()

        # Apply SG filter and detect peaks
        analyser.apply_sg_filter_and_detect_peaks(
            window_length=window_length,
            polyorder=polyorder,
            min_distance=min_distance,
            threshold=threshold
        )

        # Update Specgrid Plot
        with output_specgrid:
            clear_output(wait=True)
            try:
                # Generate specgrid plot
                fig_specgrid = analyser.plotting_specgrid(
                    bias_min, bias_max,
                    apply_gaussian_filter=apply_gaussian_filter,
                    fig_width=fig_width,
                    fig_height=fig_height,
                    save_fig=save_grid_pdf  # Save if checkbox is ticked
                )
                display(fig_specgrid)
                plt.close(fig_specgrid)
            except Exception as e:
                print(f"Error during specgrid plotting: {e}")

        # Update Average Spectrum Plot
        with output_average_spectrum:
            clear_output(wait=True)
            try:
                # Generate average spectrum plot
                fig_avg_spectrum = analyser.plotting_average_spectrum(
                    bias_min, bias_max,
                    fig_width=fig_width,
                    fig_height=fig_height,
                    save_fig=save_average_dIdV_pdf  # Save if checkbox is ticked
                )
                display(fig_avg_spectrum)
                plt.close(fig_avg_spectrum)
            except Exception as e:
                print(f"Error during average spectrum plotting: {e}")

        # Update Histogram Plot
        with output_histogram:
            clear_output(wait=True)
            try:
                # Generate histogram plot with current bias window
                fig_histogram = analyser.plot_peak_histogram(
                    bias_min=bias_min_mV,        # Pass bias window in mV
                    bias_max=bias_max_mV,
                    fig_width=fig_width,
                    fig_height=fig_height,
                    save_fig=save_histogram_pdf  # Save (if checkbox is ticked)
                )
                display(fig_histogram)
                plt.close(fig_histogram)
            except Exception as e:
                print(f"Error during histogram plotting: {e}")

        # Save Peak Histogram to CSV (if checkbox is ticked)
        if save_histogram_csv:
            try:
                analyser.save_peak_histogram_csv(bias_min=bias_min_mV, bias_max=bias_max_mV)
            except Exception as e:
                print(f"Error saving peak histogram CSV: {e}")
            finally:
                save_histogram_csv_checkbox.value = False  # Uncheck the box

        # Save Grid Output to CSV (if checkbox is ticked)
        if save_grid_csv:
            try:
                analyser.save_grid_output_csv()
            except Exception as e:
                print(f"Error saving grid output CSV: {e}")
            finally:
                save_grid_csv_checkbox.value = False  # Uncheck the box

        # Save Average dI/dV to CSV if checkbox is ticked
        if save_average_dIdV_csv:
            try:
                analyser.save_average_dIdV_csv(bias_min=bias_min, bias_max=bias_max)
            except Exception as e:
                print(f"Error saving average dI/dV CSV: {e}")
            finally:
                save_average_dIdV_csv_checkbox.value = False  # Uncheck the box

    # ============================
    # Set Up Interactive Widgets
    # ============================
    # Titles
    bias_window_title = widgets.HTML(
        value="<h3>Bias Window</h3>",
    )

    peak_detection_title = widgets.HTML(
        value="<h3>Peak Detection Settings</h3>",
    )

    figure_settings_title = widgets.HTML(
        value="<h3>Figure Settings</h3>",
    )

    saving_module_title = widgets.HTML(
        value="<h3>Saving Module</h3>",
    )

    # Organize Bias Window Controls
    bias_window_controls = widgets.VBox([
        bias_window_title,
        widgets.HBox([bias_min_input, bias_max_input]),
    ])

    # Organize Filter Options
    filter_options = widgets.VBox([
        peak_detection_title,
        widgets.HBox([window_length_slider, polyorder_slider]),
        widgets.HBox([threshold_slider, min_distance_slider]),
    ])

    # Organize Figure Settings
    figure_settings = widgets.VBox([
        figure_settings_title,
        widgets.HBox([fig_width_slider, fig_height_slider]),
    ])

    # Saving Checkboxes Layout
    saving_checkboxes = widgets.VBox([
        save_grid_csv_checkbox,
        save_average_dIdV_csv_checkbox,
        save_histogram_csv_checkbox,
        save_grid_pdf_checkbox,
        save_average_dIdV_pdf_checkbox,
        save_histogram_pdf_checkbox,
    ])

    # Organize Saving Modules
    saving_module = widgets.VBox([
        saving_module_title,
        saving_checkboxes,
    ])

    # Organize All Controls
    controls_grid = widgets.VBox([
        widgets.HBox([subtract_linear_fit_checkbox, apply_gaussian_filter_checkbox]),
        bias_window_controls,
        filter_options,
        figure_settings,
        saving_module,
    ])

    # Combine all controls and outputs
    full_layout = widgets.VBox([
        controls_grid,
        widgets.HBox([
            widgets.VBox([widgets.Label("Specgrid Plot:"), output_specgrid]),
            widgets.VBox([widgets.Label("Average Spectrum Plot:"), output_average_spectrum]),
            widgets.VBox([widgets.Label("Peak Histogram:"), output_histogram]),
        ])
    ])

    # Add observers to widgets to update plots when values change
    for widget in [
        bias_min_input, bias_max_input,
        window_length_slider, polyorder_slider,
        threshold_slider, min_distance_slider,
        apply_gaussian_filter_checkbox, subtract_linear_fit_checkbox,
        fig_width_slider, fig_height_slider,
        save_grid_csv_checkbox, save_average_dIdV_csv_checkbox, save_histogram_csv_checkbox,
        save_grid_pdf_checkbox, save_average_dIdV_pdf_checkbox, save_histogram_pdf_checkbox
    ]:
        widget.observe(update_plots, names='value')

    # Display Widgets and Output Areas
    display(full_layout)

    # Initial Plot Rendering
    update_plots()

File found: grid_data\Grid_Spectroscopy001.3ds
